<div style="text-align: center;">
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200">
</div>

# QEC Decoder Toolkit

Quantinuum offers a real-time hybrid compute capability for Quantum Error Correction (QEC) workflows. This capability executes [Web Assembly (Wasm)](https://webassembly.org/) in the H-Series stack and enables use of libraries (e.g. linear algebra and graph algorithms) and complex datastructures (e.g. vectors and graphs) during real-time execution of a quantum circuit. Unlocking experimentation of basic QEC workflows is an important milestone, as Quantinuum plans to upgrades its H-Series hardware and deploy practical QEC schemes at scale. Wasm has been used, previously, in quantum error correction schemes [[arxiv.2208.01863](https://arxiv.org/abs/2208.01863), [arxiv.2305.03828](https://arxiv.org/abs/2305.03828)]. Syndrome measurement results are sent to a classical computer where a decoding algorithm is used to determine recovery operations and update quantum circuits in real-time. 

Wasm is a binary instruction format. To compile Wasm binaries, a low-level programming language, such as Rust or C/ C++, is used to compose desired classical functions before compilation. The quantum computing workflow will then contain a call to Wasm. The folder this notebooks is in contains examples of this workflow. Wasm is used because it is a fast, safe, expressive, and easy to compile with familiar programming languages.

*[Setup and Workflow](#Setup-and-Workflow)* section includes setup instructions to setting up a python environment that can work with hybrid compute. Additional guidance is also provided on compiling Wasm binaries from Rust or C/C++ source. *[Basic Usage](#Basic-Usage)* details using TKET capabilities to link the Wasm module to a TKET circuit as well as submission to H-Series. The *[Repetition Code](#Repetition-Code)* section narrates how the Wasm feature can be used to enable a decoder-style lookup table in the Repetition code workflow.

<div style="text-align: center;">
    <img src="../figures/qec_workflow_schematic.png" width="1000">
</div>

The Mid-circuit Measurement and Reset (MCMR) with Qubit Reuse features are used in the Repetition Code workflow, which is also showcased [here](https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_mid_circuit_measurement.ipynb). An explanation of the H-Series submission workflow is found [here](https://github.com/CQCL/pytket-quantinuum/blob/develop/examples/Quantinuum_circuit_submissions.ipynb). The Wasm module is imported from TKET as well as several optional conditional operators. More information on the conditional operations available can be found in the TKET user manual ([Classical and Conditional Operations](https://tket.quantinuum.com/user-manual/manual_circuit.html#classical-and-conditional-operations)).


**Contents**
* [WebAssembly](#webassembly)
    * [Compilation](#compilation)
    * [Wasm Standards](#wasm-stanadards)
    * [FAQs](#faqs)
    * [Troubleshooting](#troubleshooting)
* [Basic Usage](#basic-usage)
* [Repetition Code](#repetition-code)
* [Surface Code](#surface-code)
* [Summary](#summary)


## WebAssembly

Wasm can be generated by composing Rust and C/ C++ code, followed by a compilation step with the Rust compiler or LLVM toolchain (Clang compiler). Once generated, the Wasm binary can be injected into the TKET workflow. The main benefit is to enable classical compute in the H-Series stack during real-time execution of the quantum circuit.

<!-- **Note:** This notebook uses existing Wasm examples provided in this folder's subdirectories. The subfolders are organized as follows.

---
    ├── Hybrid-compute              <- Examples of how to use Hybrid Compute via WASM
        |
        ├── repeat_until_success    <- Repeat Until Success source code
        ├── repetition_code         <- Repetition Code source code
        ├── surface_code            <- Surface Code source code
---
 -->

### Compilation

It is recommended to either use Rust or C++ for the compilation language to Wasm. The languages have functionality that easily compiles to Wasm. For other language options see [Wasm Getting Started Developer's page](https://webassembly.org/getting-started/developers-guide/).

<div style="text-align: center;">
    <img src="../figures/hybrid_compute_fig1.png" width="500">
</div>

The following subsections will go over:
1. Install either Rust or C/C++
1. Define the classical functions to use in your program
1. Compile to Wasm
1. Use Compiled Wasm

<table>
<tr>
    <th></th>
    <th style="text-align: center;">Rust</th>
    <th style="text-align: center;">C with Clang Toolchain</th>
</tr>
<tr style="vertical-align: text-top;">
    <td>Installation</td>
    <td>Install Rust using the <a href="https://www.rust-lang.org/tools/install">Rust Installer</a>. <br><br><b>Windows Users:</b> Before running the installer, you'll need to install the <a href="https://visualstudio.microsoft.com/visual-cpp-build-tools/">Microsoft C++ Build Tools</a>.<br><br>If you're new to Rust, see the <a href="https://doc.rust-lang.org/book/title-page.html">The Rust Programming Language</a> book. Note that Cargo is the Rust package manager. For more info on Cargo, see <a href="https://doc.rust-lang.org/cargo/guide/">The Cargo Book</a>.<br><br>If using Visual Studio code, you can also install the `rust-analyzer` extension in the Extensions tab.</td>
    <td>For Windows 10/11 users, instruction should be followed <a href="https://learn.microsoft.com/en-us/cpp/build/clang-support-msbuild?view=msvc-170">here</a> to install clang.<br><br>For Ubuntu 20.04 users, the development tools group needs to be installed with Aptitude. The LLVM and Clang compilers need to be installed separately.<br><code>sudo apt-get install build-essential<br>sudo apt-get install clang<br>sudo apt install lld-14<br></code><br>MacOS users must install XCode command line tools to enable clang usage.<br><code>xcode-select --install</code></td>
</tr>
<tr style="vertical-align: text-top;">
    <td>Project Setup</td>
    <td>Navigate to wherever you plan to store your Rust project files, and run:<br><code>cargo new --lib rus_wasm</code>.<br><br>The project name rus_wasm can be whatever project name you want to use.<br><br>This creates a new library in a subdirectory with the project name you used with everything you need to get going.<ul><li>The <i>src/lib.rs</i> file contains some generated Rust code. This code can be erased and replaced with code you plan to use in your hybrid compute program.</li><li>Inside this subdirectory is a file called <i>cargo.toml</i>. This file will be used to configure your build.</li></ul>
    <td>Navigate to wherever you plan to store your C/ C++ project files. Create a src folder containing a *.c source file. The Wasm binary that is generated, exists one-level higher than the src directory.</td>
</tr>
<tr style="vertical-align: text-top;">
    <td>Function Implementation</td>
    <td colspan="2">Examples of the types of functions that can be put in Rust or C code are provided in the respective folders within each of the example folders.<br><br>Define the classical functions you plan to use in your hybrid compute program. These will be the classical functions that your quantum program will run during execution.<ul><li>In Rust, edit the src/lib.rs file.</li><li>For C/ C++, edit the src/lib.c file. The standard C/ C++ library cannot be used in a program that is intended to be compiled to Wasm.</li></ul> </td>
</tr>
<tr style="vertical-align: text-top;">
    <td>Unit Tests</td>
    <td></td>
    <td></td>
</tr>
<tr style="vertical-align: text-top;">
    <td>Wasm Compilation</td>
    <td>Configure the <i>cargo.toml</i> file. You can use one of the examples provided to modify yours. <ol><li>Navigate to your Rust project directory from the command line.</li><li>Run: <code>cargo build</code>.<ul><li>If successful, a <i>target</i> folder and <i>cargo.lock</i> file will be created, alongside the output of the command displaying <i>Finished</i>.</li><li>If failed, Cargo will output a messaging related to the reason.</li></ul><li>Run: <code>rustup target add wasm32-unknown-unknown</code>. This allows Rust to used Wasm as a compilation target. This only needs to be run once for your environment. If you have run this previously when compiling Rust to Wasm, you do not need to run it again.</li><li>Run: <code>cargo build --release --target wasm32-unknown-unknown</code>. This compiles your Rust code to Wasm.</li></ol></td>
    <td>The C/C++ compilation uses the LLVM toolchain and requires that no standard headers be used within the program. The compilation can be invoked with the clang (clang++) compiler with specific options to compile <i>lib.c</i> (<i>lib.cpp</i>) into a Wasm binary, <i>lib.wasm</i>:<br><code>clang --target=wasm32 --no-standard-libraries -Wl,--no-entry -Wl,--export-all -o lib.wasm lib.c</code><br><br>The following compiler options are used upon Clang invocation:<ul><li><code>--target=wasm32</code>: Specify the target output as 32-bit Wasm binary.</li><li><code>--target=wasm32</code>: Specify the target output as 32 bit Wasm binary.</li><li><code>--no-standard-libraries</code>: A compiler option to prevent usage of <i>libc</i>.</li><li><code>--no-standard-libraries</code>: A compiler option to prevent usage of <i>libc</i>.</li><li><code>-Wl,--no-entry</code>: A linker option that disables the check for a main function in the c source.</li><li><code>-Wl,--export-all</code>:</li></ul></td>
</tr>
<tr style="vertical-align: text-top">
    <td>Post-Compilation Step</td>
    <td>In order to use the compiled Wasm from your quantum program, you will want to find the associated `.wasm` file that the compilation step created. With Rust, this can be found in the <i>./target/wasm32-unknown-unknown/release</i> folder of your project. You'll note a file with the name of your project with the extension <i>*.wasm</i>. To use this in your quantum program, you can copy-paste it into another folder, or ensure that the file path for your Wasm file handler points to this location.</td> 
    <td>The Wasm is generated in the directory specified when calling the clang compiler.</td>
</tr>
</table>

### Wasm Standards

There are specific constraints that must be satisfied before Wasm binaries can be submitted for execution to the real-time hybrid-compute environment. The table below specifies these constraints in more detail.

| Requirements         | Description |
|----------------------|-------------|
| Program Size         | The combined TKET + Wasm program must be less than 6 MBs. |
| C Source        | Standard C library not supported (WASI breaks the Wasm sandbox). |
| Rust Source          | Rust standard library is supported. |
| Input Arguments     | <ul><li>multiple 32-bit integers</li><li>No arguments</li></ul>|
| Method Signatures   | <ul><li>integer function (one 32-bit integer)</li><li>void function</li></ul> |
| Mandatory Functions   | <ul><li>A void function, named `init`, with zero input arguments is required to initialize the Wasm environment.</li><li>A reset function to reset in-memory Wasm variables. This function cannot be called *reset* to avoid overriding the existing *reset* function on the H-Series chip. It is recommended to add a suffix to the Wasm reset function, `reset_{variable_name}` or `reset_variables`, where `{variable_name}` is the name of the in-memory Wasm variable.</li></ul> |
| Other Restrictions    | <ul><li>RNGs, File I/O, and Exception handling not supported.</li><li>Timing restrictions (~2 milliseconds).</li></ul>|

### FAQS

**Why is Wasm useful for my use-case?**

WebAssembly enables the execution of classical logic during the real-time execution of a quantum circuit. This is useful for QEC workflows with look-up style decoders. The quantum circuit can apply Wasm on a specific classical register to perform processing on the register value. The output of the Wasm function can be stored in the same register, or passed to new register. 

**Why can I not use a higher-level programming language, such as Julia?**

There are two requirement: (1) the higher-level language is has support for Wasm compilation, and (2) the compiled Wasm must be fast and execute in ~2 miliseconds. This is best achieved with low-level languages like C and Rust. A higher-level language, such as Julia, may not generate a fast Wasm executable, despite its user-friendly nature.


**Is it possible for me to submit arbitrary Wasm to H-Series? Is this dangerous?**

No, it is not possible to submit arbitrary Wasm to H-Series. Each Wasm function must satisfy a Wasm standard. Each function is either a void function or a 32-bit integer function. For 32-bit integer Wasm function, the function must return only one 32-bit integer. The Wasm function can only accept (multiple) 32-bit integer arguments, or no arguments. 

The Wasm sandbox is an isolated memory-safe environment and communicates via network call with the H-System chip. This help alleviate the danger of end-users submitting customized Wasm binaries.

**How do I prevent qubit's idling and protect my program from memory error when using Wasm?**

For Wasm calls during circuit execution, it is recommended to use *async* Wasm calls. This is the use of void Wasm call to process the value of a register and to store some desired value as an in-memory Wasm variable. After the circuit, or subcircuit, has completed execution, the in-memory Wasm variable is retrieved using an inetegr function, and applied to the specified classical register. For in-memory Wasm variables, a Wasm call is required at the end of each shot to restore the in-memory Wasm variable to a *null* value.

**Can I also use Wasm with the H-Series emulator?**

Yes, Wasm is available for use on both the H-Series hardware and emulator.


**Can I use C++ to compile Wasm?**

Yes, clang++ can be used to compile C++ source into a Wasm binary. Identical to C, the C++ standard library cannot be used.

### Troubleshooting

## Basic Usage

To inject Wasm into a TKET program and enable processing on the H-Series devices or emulators, three components must be incorporated in the workflow.

1. [pytket.wasm.WasmFileHandler](https://tket.quantinuum.com/api-docs/wasm.html#pytket.wasm.WasmFileHandler): Enables addition of the Wasm module to the TKET program. Additionally, checks Wasm binary is compatible with Wasm standards (number of input parameters and input/ output parameter types) via a boolean kwarg, `check_file`.
1. [pytket.circuit.Circuit.add_wasm_to_reg](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.add_wasm_to_reg): Adds the linked Wasm to the TKET program as a classical operation to act across specified registers.
1. [pytket.circuit.Circuit.add_wasm](https://tket.quantinuum.com/api-docs/circuit_class.html#pytket.circuit.Circuit.add_wasm): Adds the linked Wasm to the TKET program as a classical operation to act on specific bits.
1. [wasm_file_handler](https://tket.quantinuum.com/extensions/pytket-quantinuum/api.html#pytket.extensions.quantinuum.QuantinuumBackend.process_circuits) kwarg on `QuantinuumBackend.process_circuits`: The linked Wasm module must be specified during job submission to H-Series.

Components 1 and 2 are used in the *Job Construction* subsection. Component 3 is used in the *Job Submission* subsection. 

Usage of Wasm is showcased in a Repeat Until Success (RUS) example. The RUS protocol is a non-deterministic decomposition technique to approximate a 1-qubit unitary within an allowable error threshold [[arxiv.1311.1074](https://arxiv.org/abs/1311.1074)]. The RUS circuit is composed of multiple subcircuits and each subcircuit runs successively if a specific condition is satisfied. An example is illustrated adding Wasm operations to a `pytket.circuit.Circuit` instance. Each Wasm operation calls a classical function from the linked Wasm binary.

A 2-qubit circuit is initialized. Subsequently, the `add_c_register` is used to add 4 classical registers. The first argument defines the name of the classical register. The second argument defines the size of the classical register. The following classical registers are added manually:

* `creg0` with 1 bit
* `creg1` with 1 bit
* `cond` with 32 bits
* `count` with 32 bits

In [ ]:
from pytket.circuit import Circuit, Qubit
from pytket.circuit.logic_exp import reg_lt

circuit = Circuit(2, name=f"RUS Circuit")

# Add classical registers
creg0 = circuit.add_c_register("creg0", 1)
creg1 = circuit.add_c_register("creg1", 1)
cond = circuit.add_c_register("cond", 32)
count = circuit.add_c_register("count", 32)

The `pytket.wasm.WasmFileHandler` instance is used to link to the Wasm module.

In [ ]:
import pathlib
from pytket.wasm import WasmFileHandler

rus_dir = pathlib.Path().cwd().joinpath("repeat_until_success")
wasm_file = rus_dir.joinpath("rus.wasm")
wfh = WasmFileHandler(wasm_file)

print(repr(wfh))

The linked Wasm module is added to `pytket.circuit.Circuit` instance using `add_wasm_to_reg`. The Wasm method `add_count` is called and the bit registers `creg1` and `count` are supplied. The output of `add_count` is written to `cond`. All the operations in this block are conditional operations on the decimal value of the  bit register `cond`.

In [ ]:
n_repetitions = 5
cond_execute = 3

for loop_iter in range(1, n_repetitions + 1):
    circuit.H(0, condition=reg_lt(cond, cond_execute))
    circuit.CX(0, 1, condition=reg_lt(cond, cond_execute))
    circuit.Measure(Qubit(1), creg1[0], condition=reg_lt(cond, cond_execute))

    # Call Wasm
    circuit.add_wasm_to_reg(
        "add_count", wfh, [creg1, count], [cond], condition=reg_lt(cond, cond_execute)
    )

    circuit.add_c_setreg(loop_iter, count, condition=reg_lt(cond, cond_execute))
    circuit.Reset(0, condition=reg_lt(cond, cond_execute))

circuit.Measure(Qubit(0), creg0[0])
circuit.Measure(Qubit(1), creg1[0])

A `QuantinuumBackend` instance is instantiated with the backend target specified as a H-Series emulator, `H2-1E`.

In [ ]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend("H2-1E")
backend.login();

The RUS circuit is compiled to the H-Series gate set.

In [ ]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=2)

In [ ]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(circuit)

The RUS circuit is submitted to the H-Series emulator using `process_circuits`. The `wasm_file_handler` keyword argument is used to submit the linked Wasm module to H-Series as well.

In [ ]:
handle = backend.process_circuit(compiled_circuit, n_shots=100, wasm_file_handler=wfh)

The status of the circuit can be queried with the `circuit_status` method.

In [ ]:
status = backend.circuit_status(handle)
print(status)
print(status.message)

Once the job result is ready `get_result` can be used to retrieve the data.

In [ ]:
result = backend.get_result(handle)

In [ ]:
distribution = result.get_counts(creg1)
logical_fidelity = distribution.get((0,)) / sum(distribution.values())
print(logical_fidelity)

## Repetition Code

The classical operations on bits and bit registers using `pytket.circuit.logic_exp` can be replaced with Wasm.

In [13]:
from pathlib import Path
from pytket.wasm import WasmFileHandler

wasm_file_path = Path().cwd().joinpath("repetition_code") / "c" / "rc.wasm"
wasm_file_handler = WasmFileHandler(wasm_file_path, check_file=True)

In [18]:
from pytket.circuit import Circuit, CircBox

def get_syndrome_extraction_box() -> CircBox:
    r"""This generates a syndrome extraction 
    primitive with two data qubits and one 
    ancilla qubit. The first qubit, q[0], 
    is the ancilla qubit. q[1] and q[2] are 
    data qubits.
    :returns: CircBox
    """
    circuit = Circuit(3, 1)
    circuit.name = "syndrome_extraction"
    circuit.CX(1, 0)
    circuit.CX(2, 0)
    circuit.Measure(0, 0)
    circuit.Reset(0)
    return CircBox(circuit)

syndrome_extraction = get_syndrome_extraction_box()

The `add_wasm_to_reg` method is used to apply the `decode3` function, defined in the Wasm module, to the bit registers, `syn` and `pfu`. The output of `decode3` is written to the bit register, `pfu`.

In [35]:
def get_feedforward_box() -> CircBox:
    circuit = Circuit(3)
    circuit.name = "Conditional Not"
    creg = circuit.add_c_register("creg", 3)
    for i in range(3):
        circuit.X(circuit.qubits[i], condition_bits=[creg[i]], condition_value=1);
    return CircBox(circuit)

feedforward_box = get_feedforward_box()

Classical expressions can be applied to classical bits using the `pytket.circuit.logic_exp` submodule. These expressions perform boolean operations on classical bit registers.

The `QubitRegister` and `BitRegister` objects are used to allocate qubits and bits to the `pytket.circuit.Circuit` instance. The circuit contains two quantum registers, `q` and `a`, and three classical registers:

* `c`
* `pfu`
* `syn`

The cell below adds the necessary operations to measure the stabilizers $\hat{Z}_0 \hat{Z}_1$ and $\hat{Z}_1 \hat{Z}_2$. Three measurement operations are applied to qubits within `qreg`. The outcome of the measurements is stored in the corresponding bits within `creg`, i.e. a measurement on `qreg[0]` will have a measurement outcome stored in `creg[0]`. Three Measurement operations are added to each qubit in `qreg` and the corresponding bit in `creg`.

In [47]:
from pytket.circuit import Circuit, QubitRegister, BitRegister

circuit = Circuit()
circuit.name = "repetition-code"

qreg = QubitRegister("q", 3)
circuit.add_q_register(qreg)

areg = QubitRegister("a", 1)
circuit.add_q_register(areg)

creg = BitRegister("c", 3)
circuit.add_c_register(creg)

pfu = BitRegister("pfu", 3)
circuit.add_c_register(pfu)

syn = BitRegister("syn", 2)
circuit.add_c_register(syn);

circuit.add_circbox(syndrome_extraction, [areg[0], qreg[0], qreg[1], syn[0]])
circuit.add_circbox(syndrome_extraction, [areg[0], qreg[1], qreg[2], syn[1]])
circuit.add_wasm_to_reg("decode3", wasm_file_handler, [syn], [pfu])
circuit.add_circbox(feedforward_box, list(qreg) + list(pfu))
circuit.add_circbox(syndrome_extraction, [areg[0], qreg[0], qreg[1], syn[0]])
circuit.add_circbox(syndrome_extraction, [areg[0], qreg[1], qreg[2], syn[1]])
circuit.add_wasm_to_reg("decode3", wasm_file_handler, [syn], [pfu])
circuit.add_circbox(feedforward_box, list(qreg) + list(pfu))
for i in range(3):
    circuit.Measure(qreg[i], creg[i])
circuit.add_wasm_to_reg("set_global_syn_old", wasm_file_handler, [], []);

In [48]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(circuit)

The repetition code circuit with the Wasm operation can be submitted to the `H1-1E` emulator using the usual workflow: compilation, costing, submission, status check and, finally, job retrieval. The linked Wasm module must be specified during job submission.

In [45]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend(device_name="H1-1E")
backend.login()

In [49]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=2)
render_circuit_jupyter(compiled_circuit)

In [58]:
n_shots = 1000

handle = backend.process_circuit(
    compiled_circuit, n_shots=n_shots, wasm_file_handler=wasm_file_handler
)

In [61]:
backend.circuit_status(handle)

CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "repetition-code", "submit-date": "2024-09-21T12:46:01.962271", "result-date": "2024-09-21T12:46:10.953490", "queue-position": null, "cost": "38.8", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [62]:
result = backend.get_result(handle)

In [63]:
result.get_distribution(creg)

{(0, 0, 0): 0.991,
 (0, 0, 1): 0.003,
 (0, 1, 0): 0.002,
 (0, 1, 1): 0.001,
 (1, 0, 0): 0.003}

The logical outcome is the parity of the data qubits after measurement and correction. For example, if the register `creg` is 111, then the raw outcome is 1^1^1 = 1.

1. If the correction is 000 (000^111 = 111), after correction the logical outcome is 1^1^1 = 1
2. If the correction is 010 (010^111 =  101), after correction the logical outcome is 1^0^1 = 0

If a logical zero state is prepared, then in the example above, the raw outcome is incorrect and the first corrected outcome is also incorrect (logical failure). The second corrected outcome is correct. 
 
My logical fidelity is, $\frac{N_{success}}{N_{shots}}$, where $N_{success}$ is the number of successfully corrected outcomes and $N_{shots}$ is the number of shots total.

In [64]:
import numpy as np
import pandas as pd

from pytket.backends.backendresult import BackendResult
from pytket.circuit import BitRegister


def logical_error(
    result: BackendResult,
    n_shots: int,
    classical_register: BitRegister,
    pauli_frame_register: BitRegister,
) -> pd.DataFrame:
    raw0_tot = 0
    cor0_tot = 0
    raw1_tot = 0
    cor1_tot = 0

    raw_meas = result.get_shots(creg)
    corrs = result.get_shots(pfu)
    for i in range(0, len(raw_meas)):
        raw_log = sum(raw_meas[i])%2
        corr_log = (raw_log +sum(corrs[i]))%2
        if (raw_log == 0):
            raw0_tot += 1
        if (raw_log == 1):
            raw1_tot += 1
        if (corr_log == 0):
            cor0_tot += 1
        if (corr_log == 1):
            cor1_tot += 1

    raw0_tot = raw0_tot / n_shots
    raw1_tot = raw1_tot / n_shots
    cor0_tot = cor0_tot / n_shots
    cor1_tot = cor1_tot / n_shots



    return pd.Series(
        [
            ["Raw Zeros", raw0_tot],
            ["Raw Ones", raw1_tot],
            ["Corrected Zeros", cor0_tot],
            ["Corrected Ones", cor1_tot],
        ]
    )

In [65]:
logical_error(result, n_shots, creg, pfu).head()

0          [Raw Zeros, 0.992]
1           [Raw Ones, 0.008]
2    [Corrected Zeros, 0.992]
3     [Corrected Ones, 0.008]
dtype: object

## Surface Code

Surface Code Description (*irfan and shival to add more details here.*)

This use-case demonstrates asynchronous (async) Wasm calls for a 9-qubit surface-code problem with 8-ancillary qubits. During execution of the circuit, a Wasm call uses the syndrome measurements to define an in-memory Wasm variable, `PFU` (Pauli Frame Update), via a decoder look-up table. Towards the end of a circuit, an integer Wasm function is used to retrieve the in-memory Wasm variable and perform conditional Pauli-$Z$ operations select qubits. The benefit of async calls is ability to accumulate Wasm results until they are needed for conditional operations, which helps minimize the impact of noise on a job during machine operation.

The code-cell below extracts the $Z$-syndrome. The circuit to perform this is defined over 9-qubits and encapsulated within a `pytket.circuit.CircBox`.

In [95]:
from pytket.circuit import Circuit, CircBox

def extract_z_syndrome() -> CircBox:
    circuit = Circuit()
    a = circuit.add_q_register("a", 4)
    q = circuit.add_q_register("q", 9)
    c = circuit.add_c_register("c", 4)
    
    circuit.CX(q[1], a[0])
    circuit.CX(q[0], a[0])
    circuit.Measure(a[0], c[0])

    circuit.add_barrier([a[1], q[2], q[1], q[5], q[4]])
    circuit.CX(q[2], a[1])
    circuit.CX(q[1], a[1])
    circuit.CX(q[5], a[1])
    circuit.CX(q[4], a[1])
    circuit.Measure(a[1], c[1])
    circuit.add_barrier([a[1], q[2], q[1], q[5], q[4]])

    circuit.add_barrier([a[2], q[4], q[3], q[7], q[6]])
    circuit.CX(q[4], a[2])
    circuit.CX(q[3], a[2])
    circuit.CX(q[7], a[2])
    circuit.CX(q[6], a[2])
    circuit.Measure(a[2], c[2])
    circuit.add_barrier([a[2], q[4], q[3], q[7], q[6]])


    circuit.CX(q[8], a[3])
    circuit.CX(q[7], a[3])
    circuit.Measure(a[3], c[3])
    return CircBox(circuit)

The code-cell below extracts the $X$-syndrome. The circuit to perform this is defined over 9-qubits and encapsulated within a `pytket.circuit.CircBox`.

In [96]:
def extract_x_syndrome() -> CircBox:
    circuit = Circuit()
    a = circuit.add_q_register("a", 4)
    q = circuit.add_q_register("q", 9)
    c = circuit.add_c_register("c", 4)
    
    for qubit in a:
        circuit.H(qubit)
    circuit.CX(a[0], q[2])
    circuit.CX(a[0], q[5])
    circuit.H(a[0])
    circuit.Measure(a[0], c[0])

    circuit.add_barrier([a[1], q[5], q[8], q[4], q[7]])
    circuit.CX(a[1], q[5])
    circuit.CX(a[1], q[8])
    circuit.CX(a[1], q[4])
    circuit.CX(a[1], q[7])
    circuit.H(a[1])
    circuit.Measure(a[1], c[1])
    circuit.add_barrier([a[1], q[5], q[8], q[4], q[7]])

    circuit.add_barrier([a[2], q[1], q[4], q[0], q[3]])
    circuit.CX(a[2], q[1])
    circuit.CX(a[2], q[4])
    circuit.CX(a[2], q[0])
    circuit.CX(a[2], q[3])
    circuit.H(a[2])
    circuit.Measure(a[2], c[2])
    circuit.add_barrier([a[2], q[0], q[3], q[7], q[6]])

    circuit.CX(a[3], q[3])
    circuit.CX(a[3], q[6])
    circuit.H(a[3])
    circuit.Measure(a[3], c[3])
    return CircBox(circuit)

The code-cell below applies Pauli-$X$ destabilizers to the circuit based on the value of a specified 4-bit classical register. The circuit to perform this is defined over 9-qubits and 4-bits and encapsulated within a `pytket.circuit.CircBox`.

In [97]:
def apply_x_destabillizers() -> CircBox:
    circuit = Circuit()
    q = circuit.add_q_register("q", 9)
    syn = circuit.add_c_register("syn", 4)
    circuit.X(q[1], condition_bits=[syn[0]], condition_value=1)
    circuit.X(q[2], condition_bits=[syn[0]], condition_value=1)
    circuit.X(q[2], condition_bits=[syn[1]], condition_value=1)
    circuit.X(q[7], condition_bits=[syn[2]], condition_value=1)
    circuit.X(q[8], condition_bits=[syn[2]], condition_value=1)
    circuit.X(q[8], condition_bits=[syn[3]], condition_value=1)
    return CircBox(circuit)

The code-cell below returns an $N$-qubit `CircBox` instance that applies the `OpType.Reset` operation to a specified number of qubits.

In [98]:
def reset_operations(
    n_qubits: int
) -> CircBox:
    circuit = Circuit(n_qubits)
    for qubit in circuit.qubits:
        circuit.Reset(qubit)
    return CircBox(circuit)

The code-cell below returns an $N$-qubit `CircBox` instance that applies the `Optype.H` gate to a specified number of qubits.

In [99]:
def hadamard_operations(
    n_qubits: int
) -> CircBox:
    circuit = Circuit(n_qubits)
    for qubit in circuit.qubits:
        circuit.H(qubit)
    return CircBox(circuit)

The method below generates a `CircBox` instance that applies conditional Pauli-$Z$ operations. The input parameter, `n_bits`, is used to control three registers:

* the size of the qubit register
* the size of a first classical register
* the size of a second classical register

In [100]:
def apply_x_correction(n_bits: int) -> CircBox:
    circuit = Circuit()
    q = circuit.add_q_register("q", n_bits)
    bits0 = circuit.add_c_register("bits0", n_bits)
    for i, c in enumerate(list(bits0)):
        circuit.Z(q[i], condition_bits=[c], condition_value=1)
    return CircBox(circuit)

The `CircBox` instances are instantiated below.

In [101]:
extract_z_syndrome_box = extract_z_syndrome()
extract_x_syndrome_box = extract_x_syndrome()
apply_x_destabillizers_box = apply_x_destabillizers()
hadamard_operation_box = hadamard_operations(9)
reset_operation_box = reset_operations(4)
apply_x_correction_box = apply_x_correction(9)

The wasm binary is loaded below using the `WasmFileHandler` object.

In [102]:
from pathlib import Path
from pytket.wasm import WasmFileHandler

wasm_path = Path("surface_code/rust/surface_code.wasm")
if wasm_path.is_file():
    wasm_file_handler = WasmFileHandler(wasm_path)
else:
    raise FileNotFoundError(f"{wasm_path} not found")

In [103]:
wasm_file_handler

Functions in wasm file with the uid 6750fb711d47ea7e7ae54599718f08e5b3175b8849d17db086cd724ff503672b:
function 'init' with 0 i32 parameter(s) and 0 i32 return value(s)
function 'set_pfu_value' with 1 i32 parameter(s) and 0 i32 return value(s)
function 'update_pfu' with 1 i32 parameter(s) and 0 i32 return value(s)
function 'get_pfu' with 0 i32 parameter(s) and 1 i32 return value(s)
function 'reset_pfu' with 0 i32 parameter(s) and 0 i32 return value(s)

The circuit is constructed by adding the `CircBox` instances, as shown below.

In [104]:
from pytket.circuit import Circuit

circuit = Circuit()
ax = circuit.add_q_register('ax', 4)
az = circuit.add_q_register('az', 4)
q = circuit.add_q_register('q', 9)
syn_x = circuit.add_c_register('syn_x', 4)
syn_z = circuit.add_c_register('syn_z', 4)
pfu = circuit.add_c_register("pfu", 9)
creg = circuit.add_c_register("creg", 9)

circuit.add_circbox(hadamard_operation_box, list(q))
circuit.add_circbox(extract_z_syndrome_box, list(az) + list(q) + list(syn_z))
circuit.add_circbox(reset_operation_box, list(az))
circuit.add_circbox(apply_x_destabillizers_box, list(q) + list(syn_z))
circuit.add_circbox(extract_x_syndrome_box, list(ax) + list(q) + list(syn_x))
circuit.add_wasm_to_reg("set_pfu_value", wasm_file_handler, [syn_x], [])
circuit.add_circbox(reset_operation_box, list(ax))
circuit.add_circbox(extract_x_syndrome_box, list(ax) + list(q) + list(syn_x))
circuit.add_wasm_to_reg("update_pfu", wasm_file_handler, [syn_x], [])
circuit.add_wasm_to_reg("get_pfu", wasm_file_handler, [], [pfu])
circuit.add_circbox(apply_x_correction_box, list(q) + list(pfu))
circuit.add_circbox(hadamard_operation_box, list(q))
for qubit, bit in zip(list(q), list(creg)):
    circuit.Measure(qubit, bit)
circuit.add_wasm_to_reg("reset_pfu", wasm_file_handler, [], []);

The `QuantinuumBackend` object is constructed and the circuit is compiled and submitted using the usual workflow. The `WasmFileHandler` instance is submitted using the `kwarg` on `process_circuits`.

In [105]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend(device_name="H1-1E")
backend.login()

In [106]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=2)

In [107]:
handle = backend.process_circuit(
    compiled_circuit, n_shots=100, wasm_file_handler=wasm_file_handler)

In [109]:
backend.circuit_status(handle)

CircuitStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='{"name": "job", "submit-date": "2024-09-21T13:01:52.969485", "result-date": "2024-09-21T13:02:27.981698", "queue-position": null, "cost": "18.68", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [110]:
result = backend.get_result(handle)

The shots table for the `creg` registers is retrieved below. The post-processing should show all syndrome measurements are zero after correction.

In [112]:
shot_table = result.get_shots(creg)

for i in range(10):
    m = shot_table[i]
    s1 = (int(m[6]) + int(m[3]))%2
    s2 = (int(m[3]) + int(m[4]) + int(m[1]) + int(m[0]))%2
    s3 = (int(m[8-0]) + int(m[8-1]) + int(m[8-3]) + int(m[8-4]))%2
    s4 = (int(m[5]) + int(m[2]))%2
    l = (int(m[8]) + int(m[7]) + int(m[6]))%2
    outcome = (s1, s2, s3, s4, l)
    majority = int(outcome.count(0) < outcome.count(1))
    print(majority)

0
0
0
0
0
0
0
0
0
0


## Summary

The hybrid compute feature enables classical compute logic compiled to a Wasm binary to be linked and added to a TKET program for processing on H-Series. The classical compute portion of the program is processed during real-time execution of the quantum circuit. This is the first major step to practical QEC at scale as H-Series hardware evolves towards fault-tolerance. The 3-qubit repetition code protocol uses Wasm to decode syndromes via a lookup table.

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>